# Bank Loan
---

1. Find best model
    - Test 4 models
    - Test 4 sampling:
        - Oversampling `SMOTE`, `RandomOverSampler`,
        - Undersampling `RandomUnderSampler`, `NearMiss`
    - Minority Nearmiss
2. Tune best model
    - 

# 1. Load Dataset & Library

In [1]:
## EDA Standard Libary

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats as ss

In [2]:
#ML Library

#ML Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#ML TrainTest Split
from sklearn.model_selection import train_test_split
#ML Report
from sklearn.metrics import  accuracy_score

In [3]:
#Load data
df = pd.read_csv('/Users/Dwika/My Projects/DATASETS/bankloan.csv') 

In [5]:
df.head()

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
0,41,3,17,12,176,9.3,11.359392,5.008608,1
1,27,1,10,6,31,17.3,1.362202,4.000798,0
2,40,1,15,14,55,5.5,0.856075,2.168925,0
3,41,1,15,14,120,2.9,2.658720,0.821280,0
4,24,2,2,0,28,17.3,1.787436,3.056564,1


In [12]:
df.describe()

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default
count,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000,700.000000
mean,34.860000,1.722857,8.388571,8.278571,45.601429,10.260571,1.553553,3.058209,0.261429
std,7.997342,0.928206,6.658039,6.824877,36.814226,6.827234,2.117197,3.287555,0.439727
min,20.000000,1.000000,0.000000,0.000000,14.000000,0.400000,0.011696,0.045584,0.000000
25%,29.000000,1.000000,3.000000,3.000000,24.000000,5.000000,0.369059,1.044178,0.000000
50%,34.000000,1.000000,7.000000,7.000000,34.000000,8.600000,0.854869,1.987567,0.000000
75%,40.000000,2.000000,12.000000,12.000000,55.000000,14.125000,1.901955,3.923065,1.000000
max,56.000000,5.000000,31.000000,34.000000,446.000000,41.300000,20.561310,27.033600,1.000000


# 2. Preprocessing

In [8]:
#Check balance of default
df['default'].value_counts()

default
0    517
1    183
Name: count, dtype: int64

## Create Transformer Schema

In [11]:
#Scaling & Encoding Library
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer


#Null Imputer
from sklearn.impute import SimpleImputer

In [21]:
#Create transformer schema
transformer = ColumnTransformer([
    ('Scaler',RobustScaler() , ['income','debtinc','creddebt',	'othdebt']),
    ('Binary Encoder', ce.BinaryEncoder(), ['employ','address'])], remainder='passthrough'
)

In [32]:
transformer

ColumnTransformer(remainder='passthrough',
                  transformers=[('Scaler', RobustScaler(),
                                 ['income', 'debtinc', 'creddebt', 'othdebt']),
                                ('Binary Encoder', BinaryEncoder(),
                                 ['employ', 'address'])])

## Test Split with imbalance classification

In [30]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.pipeline import Pipeline

In [24]:
#Train, test split
y = df['default']
x = df.drop('default', axis=1)

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x,y, stratify=y, test_size=0.2, random_state=2020)

## Finding Beest Model with Cross Validation

In [28]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [29]:
#Cross Validation with KFold
kfold = StratifiedKFold(n_splits=5)


In [85]:
import warnings 
warnings.filterwarnings('ignore')

In [98]:
#Model Benchmarking
#---------------------------------------------
#Decision Tree
dt = DecisionTreeClassifier(
    criterion='entropy',
    class_weight='balanced',
    max_depth=5,
)

#KNN
knn = KNeighborsClassifier(
    n_neighbors=5,
    weights='distance',
    algorithm='auto'
)

#SVC
svc = SVC(
    C=1.0,
    kernel='rbf',
    degree=3,
    gamma='scale'
)

#Logistic Regression
logreg = LogisticRegression(
    penalty='l2',
    solver='lbfgs',
    C=1.0,
    max_iter=1000
)

#Create Pipeline
model_list = [dt, knn, svc, logreg]

model_scores = []
model_f1 = []
model_std = []



balance_method = [SMOTE(), RandomUnderSampler(), NearMiss()]

def ModelLoop(balancing) : 
    
    model_scores = []
    model_f1 = []
    model_std = []
    
    
    #Loop through all models
    
    
    for i in model_list:
        model_pipe = Pipeline([
            ('Preprocessing', transformer),
            ('Balance',balancing),
            ('Model', i)
        ])
        model_cv = cross_val_score(model_pipe, x_train, y_train, cv=kfold, scoring='f1')
        model_scores.append(model_cv.mean())
        model_std.append(model_cv.std())
        model_f1.append(model_cv)




In [103]:
for i in balance_method:
    ModelLoop(i)

In [104]:
model_scores

[]

In [100]:
df_model_cv = pd.DataFrame({'Model' : ['Decision Tree', 'KNN', 'SVC', 'Logistic Regression'],
                           'F1 Score Mean': model_scores,
                           'F1 Score STD':model_std})
df_model_cv.sort_values('F1 Score Mean', ascending=False)

ValueError: All arrays must be of the same length